In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy.stats import skewnorm, norm
import arviz as az

In [3]:
rng = np.random.default_rng(10)
x = skewnorm(3, loc=5, scale=2).rvs(size=50, random_state=rng)
y = - np.ones_like(x)

## Computing kde animation

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial

fig, ax = plt.subplots()
grid, kde, bw = az.stats.density_utils.kde(x, bw_return=True)
kernels = [ax.plot([], [], "-", color="lightgray")[0] for _ in x]
rugs = [ax.plot([], [], "|k", markersize=10)[0] for _ in x]
kde_artist, = ax.plot([], [], "b-", alpha=0.5, linewidth=1)
kde_boundary, = ax.plot([], [], "b-", linewidth=2)

ax.legend(
    handles=[kde_boundary, kde_artist, kernels[0], rugs[0]],
    labels=["KDE", "KDE (no boundary\ncorrection)", "Kernel", "Data"]
)

kde_cum_grid = np.linspace(grid.min() - 1, grid.max() + 1, 500)

def init():
    ax.set_title('Kernel Density Estimate (KDE)')
    ax.spines.left.set_color('none')
    ax.spines.right.set_color('none')
    ax.spines.bottom.set_position('zero')
    ax.spines.top.set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.tick_params(left=False, labelleft=False)
    ax.set_ylim(-0.1, 0.4)
    ax.set_xlim(kde_cum_grid.min(), kde_cum_grid.max())
    for k in kernels:
        k.set_data([], [])
    kde_artist.set_data([], [])
    kde_artist.set(alpha=1, linewidth=2)
    kde_boundary.set_data([], [])
    for r in rugs:
        r.set_data([], [])
    return tuple([kde_artist, kde_boundary, *kernels, *rugs])

def update(frame, samples):
    step, idx = frame
    if step == "rug":
        for rug, sample in zip(rugs, samples):
            rug.set_data([sample], [-0.05])
            rug.set_color("black")
    elif step == "kde":
        rugs[idx].set(color="red", markersize=30)
        if idx > 0:
            rugs[idx-1].set(color="lightgray", markersize=10)
            kernels[idx-1].set_color("lightgray")
        kernel = norm.pdf(kde_cum_grid, loc=samples[idx], scale=bw) / len(samples)
        _, kde_cum = kde_artist.get_data()
        if not isinstance(kde_cum, np.ndarray):
            kde_cum = np.zeros_like(kde_cum_grid)
        kde_artist.set_data(kde_cum_grid, kde_cum+kernel)
        kernels[idx].set_data(kde_cum_grid, kernel)
        kernels[idx].set(color="red")
    elif step == "cleanup":
        rugs[-1].set(color="lightgray", markersize=10)
        kernels[-1].set_color("lightgray")
    elif step == "boundary":
        if idx == "show_bounds":
            kde_artist.set(alpha=0.5, linewidth=1)
            _, kde_cum = kde_artist.get_data()
            mask = (kde_cum_grid > samples.min()) & (kde_cum_grid < samples.max())
            kde_boundary.set_data(kde_cum_grid[mask], kde_cum[mask])
            rugs[samples.argmin()].set(color="red", markersize=30)
            rugs[samples.argmax()].set(color="red", markersize=30)
        if idx == "correct":
            kde_boundary.set_data(grid, kde)
            rugs[samples.argmin()].set(color="lightgray", markersize=10)
            rugs[samples.argmax()].set(color="lightgray", markersize=10)
    return tuple([kde_artist, kde_boundary, *kernels, *rugs])

frames = [
    *[("wait", None)]*2,
    ("rug", None),
    *[("wait", None)]*4,
    *[("kde", i) for i in range(len(x))],
    ("cleanup", None),
    *[("wait", None)]*2,
    ("boundary", "show_bounds"),
    *[("wait", None)]*3,
    ("boundary", "correct"),
]

ani = FuncAnimation(
    fig, partial(update, samples=x),
    frames=frames,
    init_func=init,
    blit=True,
    interval=250,
    repeat_delay=3000,
)

ani.save("kde.mp4", dpi=300)

plt.show()

##